In [ ]:
# ✅ Install bitsandbytes (8-bit quantization support)
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 116.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0


## ✅ Step 1: Load Model & Tokenizer with 8bit Quantization

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"

quant_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

## ✅ Step 2: Apply LoRA

In [ ]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,572,864 || all params: 1,348,044,800 || trainable%: 0.1167


## ✅ Step 3: Load Dataset

In [ ]:

from datasets import load_dataset

dataset = load_dataset("json", data_files="lawyer_data.jsonl")

def format_example(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    }

dataset = dataset.map(format_example)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/501 [00:00<?, ? examples/s]

## ✅ Step 4: Tokenize

In [ ]:

def tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    response = example["output"]

    input_enc = tokenizer(prompt + response, truncation=True, padding="max_length", max_length=512)
    label_enc = tokenizer(response, truncation=True, padding="max_length", max_length=512)

    label_enc["input_ids"] = [
        token if token != tokenizer.pad_token_id else -100 for token in label_enc["input_ids"]
    ]

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": label_enc["input_ids"]
    }

dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/501 [00:00<?, ? examples/s]

## ✅ Step 5: Setup Trainer

In [ ]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="deepseek_lawyer",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer
)


/tmp/ipython-input-6-3707687449.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## ✅ Step 6: Train the Model

In [ ]:
trainer.train()

Step,Training Loss
10,3.301800
20,3.179700
30,2.660100
40,2.216900
50,1.841000
60,1.449400
70,1.159400
80,0.884200
90,0.487100
100,0.185300


TrainOutput(global_step=753, training_loss=0.23294025037040658, metrics={'train_runtime': 418.8351, 'train_samples_per_second': 3.589, 'train_steps_per_second': 1.798, 'total_flos': 5919199400558592.0, 'train_loss': 0.23294025037040658, 'epoch': 3.0})

## ✅ Step 7: Save the Model

In [ ]:

model.save_pretrained("deepseek_lawyer")
tokenizer.save_pretrained("deepseek_lawyer")


('deepseek_lawyer/tokenizer_config.json',
 'deepseek_lawyer/special_tokens_map.json',
 'deepseek_lawyer/chat_template.jinja',
 'deepseek_lawyer/tokenizer.json')

## ✅ Step 8: Inference Function for Chatbot

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek_lawyer", trust_remote_code=True)

def ask(question):
    prompt = f"### Instruction:\n{question}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = result.replace(prompt, "").strip()
    print("🤖 Lawyer Assistant:\n", response)

# Example
ask("what is law?")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


🤖 Lawyer Assistant:
 Law is a legal procedure or course that ensures the exercise of rights under a law. It includes procedures such as trial, pre-trial, post-trial, and custody. Laws can be legalized by various methods, including trial, pre-trial, post-trial, and custody.


In [ ]:
ask("what is criminal law?")

🤖 Lawyer Assistant:
 Criminal law is a legal procedure that ensures the protection of a person from being criminalized. It includes procedures for detection, sentence preparation, and custody.


In [ ]:
ask("what is FIR?")

🤖 Lawyer Assistant:
 FIR stands for First Information Report. It is a document prepared by police when they receive information about a cognizable offense.


In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load your fine-tuned model
model_path = "deepseek_lawyer"  # Or "checkpoint-xxx" if that's your best checkpoint
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

# Chat function
def generate_response(user_input):
    prompt = f"### Instruction:\n{user_input}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()
    return response

# Streamlit UI
st.set_page_config(page_title="📜 Legal Chatbot - FIR Assistant", layout="centered")
st.title("🧑‍⚖️ FIR & Law Chatbot")
st.markdown("Ask me anything about FIRs, criminal law, or general legal procedure.")

# User input
user_query = st.text_input("📝 Enter your legal question here:")

if user_query:
    with st.spinner("Thinking like a lawyer..."):
        answer = generate_response(user_query)
    st.success("🤖 " + answer)


2025-07-29 18:59:42.969 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 18:59:42.971 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 18:59:43.207 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-29 18:59:43.212 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 18:59:43.216 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 18:59:43.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 18:59:43.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [ ]:
model.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")
tokenizer.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")


('/content/drive/MyDrive/deepseek_lawyer/tokenizer_config.json',
 '/content/drive/MyDrive/deepseek_lawyer/special_tokens_map.json',
 '/content/drive/MyDrive/deepseek_lawyer/chat_template.jinja',
 '/content/drive/MyDrive/deepseek_lawyer/tokenizer.json')

In [ ]:
from transformers import AutoConfig

# Load base model's config
base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"
config = AutoConfig.from_pretrained(base_model)

# Save it inside your fine-tuned model folder
config.save_pretrained("/content/deepseek_lawyer")  # or your path


In [ ]:
!zip -r /content/drive/MyDrive/deepseek_lawyer.zip /content/drive/MyDrive/deepseek_lawyer


  adding: content/drive/MyDrive/deepseek_lawyer/ (stored 0%)
  adding: content/drive/MyDrive/deepseek_lawyer/generation_config.json (deflated 21%)
  adding: content/drive/MyDrive/deepseek_lawyer/special_tokens_map.json (deflated 69%)
  adding: content/drive/MyDrive/deepseek_lawyer/config.json (deflated 50%)
  adding: content/drive/MyDrive/deepseek_lawyer/adapter_model.safetensors (deflated 8%)
  adding: content/drive/MyDrive/deepseek_lawyer/tokenizer_config.json (deflated 88%)
  adding: content/drive/MyDrive/deepseek_lawyer/tokenizer.json (deflated 81%)
  adding: content/drive/MyDrive/deepseek_lawyer/adapter_config.json (deflated 55%)
  adding: content/drive/MyDrive/deepseek_lawyer/chat_template.jinja (deflated 57%)


In [ ]:
model.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")
tokenizer.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")
config.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")